# End-to-End Evaluations

We construct an end-to-end RAG pipeline: 
* Parse unstructured data from PDFs leveraging the raw-text extraction approach
* Employ OpenAI’s text-embedding-3-large model to index and retrieve relevant data chunks. 
* In the generation phase, we incorporate the Chain-of-Thought approach to handle arithmetic-intensive tasks.

 Based on this end-to-end pipeline, we evaluate 8 LLMs spanning various model sizes and architectures.

In [1]:
# Preparation
import sys
import os
from pathlib import Path

# Get the project root directory
root_dir = Path(os.path.abspath("")).resolve().parents[1]
sys.path.append(str(root_dir))
# Change the working directory to the project root
os.chdir(root_dir)

res_dir = f"experiment/e2e/res/"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
    
import warnings
warnings.filterwarnings('ignore')

In our paper, we use the powerful `text-embedding-3-large-model` model with AzureOpenAI-API . But you need to set up with your own api-key, endpoint and deploy-model in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). 

If you want to use the API from **other alternative platforms** please change the codes in [uda/utils/retrieve.py (line-80)](../../uda/utils/retrieve.py#L81). 

For convenient demonstration, we choose the `BM25` retriever here.

In [2]:
# Experimental Configurations

# Available retrieval model_name: "bm25", "all-MiniLM-L6-v2", "all-mpnet-base-v2", "openai", "colbert"
# We choose bm25 for convenience
RT_MODEL = "bm25" 

DATASET_NAME_LIST = ["fin", "feta", "tat","paper_text", "nq", "paper_tab"]
LOCAL_LLM_DICT = {
    "llama-8B": "meta-llama/Meta-Llama-3-8B-Instruct",
    "qwen-7B": "Qwen/Qwen1.5-7B-Chat",
    "mistral": "mistralai/Mistral-7B-Instruct-v0.2",
    "qwen-32B": "Qwen/Qwen1.5-32B-Chat",
    "mixtral": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "llama-70B": "meta-llama/Meta-Llama-3-70B-Instruct",
}
LLM_LIST = ["gpt4", "llama-8B", "mistral", "qwen-32B", "mixtral", "llama-70B", "qwen-7B"]

# Sample a subset for faster demo
DATASET_NAME_LIST = DATASET_NAME_LIST[:2]
LLM_LIST = LLM_LIST[:1]
DEMO_DOC_NUM = 2
DEMO_QA_NUM = 2

In our implementation, the **AzureOpenAI-API** serves as the interface for accessing GPT models. Users should set up the gpt-service with their own api-key and endpoint in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). These configurations will be used in the `call_gpt()` function in the following codes.


If you want to use **other alternative platforms**, the `call_gpt()` can be replaced by the corresponding model-calling function.

In [3]:
from uda.utils import retrieve as rt
from uda.utils import preprocess as pre
import pandas as pd
from uda.utils import llm
from uda.utils import inference
import json

for DATASET_NAME in DATASET_NAME_LIST:
    for LLM_MODEL in LLM_LIST:
        print(f"=== Start {DATASET_NAME} on {LLM_MODEL} ===")
        res_file = os.path.join(res_dir, f"{DATASET_NAME}_{LLM_MODEL}_{RT_MODEL}.jsonl")

        # If use the local LLM, initialize the model
        if LLM_MODEL in LOCAL_LLM_DICT:
            llm_name = LOCAL_LLM_DICT[LLM_MODEL]
            llm_service = inference.LLM(llm_name)
            llm_service.init_llm()

        # Load the benchmark data
        bench_json_file = pre.meta_data[DATASET_NAME]["bench_json_file"]
        with open(bench_json_file, "r") as f:
            bench_data = json.load(f)

        # Run experiments on the demo docs
        doc_list = list(bench_data.keys())
        for doc in doc_list[:DEMO_DOC_NUM]:
            pdf_path = pre.get_example_pdf_path(DATASET_NAME, doc)
            if pdf_path is None:
                continue
            # Prepare the index for the document
            collection_name = f"{DATASET_NAME}_vector_db"
            collection = rt.prepare_collection(pdf_path, collection_name, RT_MODEL)
            for qa_item in bench_data[doc][:DEMO_QA_NUM]:
                question = qa_item["question"]
                # Retrieve the contexts
                contexts = rt.get_contexts(collection, question, RT_MODEL)
                context_text = '\n'.join(contexts)
                # Create the prompt
                llm_message = llm.make_prompt(question, context_text, DATASET_NAME, LLM_MODEL)
                # Generate the answer
                if LLM_MODEL in LOCAL_LLM_DICT:
                    response = llm_service.infer(llm_message)
                elif LLM_MODEL == "gpt4":
                    # Set up with your own GPT4 service using environment variables
                    response = llm.call_gpt(messages=llm_message)
                    if response is None:
                        print("Make sure your gpt4 service is set up correctly.")
                        raise Exception("GPT4 service")

                # log the results
                res_dict = {"model": LLM_MODEL, "question": question, "response": response, "doc": doc, "q_uid": qa_item["q_uid"], "answers": qa_item["answers"]}
                print(res_dict)
                with open(res_file, "a") as f:
                    f.write(json.dumps(res_dict) + "\n")
            rt.reset_collection(collection_name, RT_MODEL)

    print(f"=== Finish {DATASET_NAME} ===\n")


=== Start fin on gpt4 ===


2024-07-05 12:34:41,146 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'what percentage of total long-term assets under supervision are comprised of fixed income in 2015?', 'response': 'In 2015, the total long-term assets under supervision were $946 billion, and the fixed income assets under supervision were $546 billion.\n\nTo calculate the percentage of fixed income assets in the total long-term assets under supervision for 2015:\n\nFixed income assets / Total long-term assets under supervision * 100\n= $546 billion / $946 billion * 100\n= 57.72%\n\nThe answer is: 57.72%', 'doc': 'GS_2016', 'q_uid': 'GS/2016/page_79.pdf-3', 'answers': {'str_answer': '57%', 'exe_answer': 0.57484}}


2024-07-05 12:34:47,282 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'what percentage of total long-term assets under supervision are comprised of fixed income in 2016?', 'response': 'In 2016, the total long-term assets under supervision are $1,021 billion, and the fixed income assets under supervision are $601 billion. To find the percentage of fixed income assets in the total long-term assets under supervision:\n\nPercentage = (Fixed Income Assets / Total Long-Term Assets) * 100\nPercentage = ($601 billion / $1,021 billion) * 100\nPercentage = 58.86%\n\nThe answer is: 58.86%', 'doc': 'GS_2016', 'q_uid': 'GS/2016/page_79.pdf-1', 'answers': {'str_answer': '59%', 'exe_answer': 0.588}}


...

{'model': 'gpt4', 'question': 'in 2010 , what was the cumulative total return of the s&p 500?', 'response': 'The answer is: 100.00', 'doc': 'JKHY_2015', 'q_uid': 'JKHY/2015/page_20.pdf-3', 'answers': {'str_answer': '30.69', 'exe_answer': 30.69}}
=== Finish fin ===

=== Start feta on gpt4 ===


2024-07-05 12:34:59,611 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': "Which season of Smallville performed the best during it's airing? ", 'response': 'The answer is: The first season of Smallville performed the best during its airing, with the highest rank of 115 and an average of 5.90 million viewers per episode.', 'doc': 'Smallville', 'q_uid': 12844, 'answers': "Over ten seasons the Smallville averaged, million viewers per episode, is with season two's highest rating of 6.3 million."}


2024-07-05 12:35:05,499 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'In which film did Jennifer Jones star in 1995 and in which consequent film did she take on a role in 1956? ', 'response': 'The answer is: Jennifer Jones did not star in any film in 1995 as her acting career was active from 1939 to 1974. However, in 1956, she starred in "The Man in the Gray Flannel Suit."', 'doc': 'Jennifer Jones', 'q_uid': 19050, 'answers': 'Jennifer Jones starred in Good Morning, Miss Dove in 1955, followed by a role in The Man in the Gray Flannel Suit.'}
=== Finish feta ===



### Evaluate the end-to-end results

In [5]:
dataset_name="feta"
llm_model="gpt4"
rt_model="bm25"
res_file_name=f"experiment/e2e/res/{dataset_name}_{llm_model}_{rt_model}.jsonl"

from uda.eval.my_eval import eval_from_file
eval_from_file(dataset_name, res_file_name)

{'Answer F1': 0.44500000000000006, 'Missing predictions': 0}
